## Performance of the CPL
Author: Clara Hoffmann \
Measures the performance of the CPL on the precise validation set.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy as sp
from tqdm import tqdm
from scipy.integrate import simps, trapz
import matplotlib.pyplot as plt
import imageio
import multiprocessing
from helpers import (Fy, find_closest_element,  compute_coverage, #predict_single_density,
confidence_interval, confidence_interval, generate_fixed_terms, get_ci)
from density_predictor_class_val import density_predictor

In [2]:
# read in val data
B_zeta = np.load('../../../../data/commaai/extracted_coefficients/copula_cil/Bzeta/B_zeta_val.npy')
beta = np.genfromtxt('../../../../data/commaai/extracted_coefficients/copula_cil/beta/beta.csv', delimiter = ',')

true_y = np.load('../../../../data/commaai/extracted_coefficients/copula_cil/Bzeta/labels_val.npy')
true_z = np.load('../../../../data/commaai/extracted_coefficients/copula_cil/Bzeta/tr_labels_val.npy')

B_zeta = B_zeta[np.abs(true_y) < 40,:] 
true_z = true_z[np.abs(true_y) < 40]
true_y = true_y[np.abs(true_y) < 40]

density_path = '../../../../data/commaai/density/gaussian_density.csv'
density = pd.read_csv(density_path)

no_points = 1000
y_pred = np.load('../../../../data/commaai/predictions/mc_dropout/y_preds_cil.npy' ).reshape(-1)

In [3]:
dens = density_predictor(B_zeta, true_y, density, no_points)

In [5]:
result_va_ridge = dens.get_density('va_ridge')
#result_hmc_ridge = dens.get_density('hmc_ridge')

  0%|          | 68/94570 [00:00<02:20, 673.19it/s]

computing densities for each observation


  1%|          | 519/94570 [00:00<00:18, 5183.68it/s]

computing mean prediction for each observation


  1%|          | 1147/94570 [00:00<00:08, 11465.69it/s]

computing variance prediction for each observation


100%|██████████| 94570/94570 [00:08<00:00, 11211.28it/s]


In [6]:
result_va_horse = dens.get_density('va_horseshoe')
#result_hmc_horse = dens.get_density('hmc_horseshoe')

100%|██████████| 94570/94570 [00:08<00:00, 11206.91it/s]


In [7]:
#z_pred = B_zeta.dot(beta) 
#y_pred = np.array([density.loc[find_closest_element(norm.cdf(i), density['cdf']), 'axes'] for i in tqdm(z_pred)])
y_pred = np.load('../../../../data/commaai/predictions/y_preds_cil_no_mc.npy').reshape(-1)

In [8]:
#preds_1 = np.append(np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_cil_y_1.npy'), 
#                  np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_cil_y_2.npy'), axis = 0)
#preds_mc = np.mean(preds_1, axis = 1).reshape(-1)

In [9]:
x1 = np.load('../../../../data/commaai/predictions/mc_preds_cil_1_neu.npy').reshape(-1,1000)
x2 = np.load('../../../../data/commaai/predictions/mc_preds_cil_2_neu.npy').reshape(-1,1000)
x3 = np.load('../../../../data/commaai/predictions/mc_preds_cil_3_neu.npy').reshape(-1,1000)
x4 = np.load('../../../../data/commaai/predictions/mc_preds_cil_4_neu.npy').reshape(-1,1000)

preds_1 = np.append(np.append(np.append(x1, x2, axis = 0), x3, axis = 0), x4, axis = 0)
preds_mc = np.mean(preds_1, axis = 1).reshape(-1)

In [10]:
mdn_preds = np.load('../../../../data/commaai/predictions/mdn/cil/samples.npy')
pred_mdn = np.mean(mdn_preds, axis = 1)

In [12]:
print('Validation Performance: ' +
      '\nMSE:' +
     '\n CIL VA/Ridge: ' + str(np.mean((true_y - np.array(result_va_ridge['mean predictions']))**2)) +
     #'\n CIL HMC/Ridge: ' + str(np.mean((true_y - np.array(result_hmc_ridge['mean prediction']))**2)) +
     '\n CIL VA/Horseshoe: ' + str(np.mean((true_y - np.array(result_va_horse['mean prediction']))**2)) +
     #'\n CIL HMC/Horseshoe: ' + str(np.mean((true_y - np.array(result_hmc_horse['mean prediction']))**2)) +
     '\n MC-Dropout: ' + str(np.mean((true_y - preds_mc)**2)) +
     '\n MDN: ' + str(np.mean((true_y - pred_mdn)**2)) +
     '\n CIL No Bayes: ' + str(np.mean((true_y - y_pred)**2)) +
     '\n CIL Benchmark: ' + str(np.mean((true_y - 0)**2)) +
      
    '\nMAE:' +
     '\n CIL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])))) +
     #'\n CIL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])))) +
     '\n CIL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])))) +
     #'\n CIL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])))) +
      '\n MDN: ' + str(np.mean(np.abs(true_y - pred_mdn))) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc))) +
     '\n CIL No Bayes: ' + str(np.mean(np.abs(true_y - y_pred))) +
     '\n CIL Benchmark: ' + str(np.mean(np.abs(true_y - 0))) +
      
      '\nAccuracy I:' +
     '\n CIL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) <= 6)) +
     #'\n CIL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) <= 6)) +
     '\n CIL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) <= 6)) +
     #'\n CIL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])) <= 6)) +
     '\n MDN: ' + str(np.mean(np.abs(true_y - pred_mdn) <= 6)) +
     '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc) <= 6)) +
      '\n CIL No Bayes: ' + str(np.mean(np.abs(true_y - y_pred) <= 6)) +
     '\n CIL Benchmark: ' + str(np.mean(np.abs(true_y - 0) <= 6)) +
    
      '\nAccuracy II:' +
     '\n CIL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) <= 2)) +
     #'\n CIL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) <= 2)) +
     '\n CIL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) <= 2)) +
     #'\n CIL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])) <= 2)) +
    '\n MDN: ' + str(np.mean(np.abs(true_y - pred_mdn) <= 2)) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc) <= 2)) +
     '\n CIL No Bayes: ' + str(np.mean(np.abs(true_y - y_pred) <= 2)) +
     '\n CIL Benchmark: ' + str(np.mean(np.abs(true_y - 0) <= 2)) 
     )

Validation Performance: 
MSE:
 CIL VA/Ridge: 10.564505277332286
 CIL VA/Horseshoe: 10.08426034928085
 MC-Dropout: 11.46415893298916
 MDN: 9.199817722941372
 CIL No Bayes: 11.91994377532943
 CIL Benchmark: 14.780700856508409
MAE:
 CIL VA/Ridge: 1.8820454127620942
 CIL VA/Horseshoe: 1.8104470725771844
 MDN: 1.5866840416764973
 MC-Dropout: 1.9341478468806148
 CIL No Bayes: 1.9613557251596458
 CIL Benchmark: 2.3610827958126257
Accuracy I:
 CIL VA/Ridge: 0.9526594057312044
 CIL VA/Horseshoe: 0.9571005604314264
 MDN: 0.9658559796975785
 MC-Dropout: 0.9533361531140954
 CIL No Bayes: 0.9473828909802263
 CIL Benchmark: 0.9205244792217405
Accuracy II:
 CIL VA/Ridge: 0.7016918684572274
 CIL VA/Horseshoe: 0.7121603045363223
 MDN: 0.7728666596172148
 MC-Dropout: 0.6989531563920905
 CIL No Bayes: 0.6986042085227874
 CIL Benchmark: 0.5965422438405414


In [ ]:
print('Validation Performance: ' +
      '\nMSE:' +
     '\n CIL VA/Ridge: ' + str(np.mean((true_y - np.array(result_va_ridge['mean predictions']))**2)) +
     #'\n CIL HMC/Ridge: ' + str(np.mean((true_y - np.array(result_hmc_ridge['mean prediction']))**2)) +
     '\n CIL VA/Horseshoe: ' + str(np.mean((true_y - np.array(result_va_horse['mean prediction']))**2)) +
     #'\n CIL HMC/Horseshoe: ' + str(np.mean((true_y - np.array(result_hmc_horse['mean prediction']))**2)) +
     '\n MC-Dropout: ' + str(np.mean((true_y - preds_mc)**2)) +
     '\n MDN: ' + str(np.mean((true_y - pred_mdn)**2)) +
     '\n CIL No Bayes: ' + str(np.mean((true_y - y_pred)**2)) +
     '\n CIL Benchmark: ' + str(np.mean((true_y - 0)**2)) +
      
    '\nMAE:' +
     '\n CIL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])))) +
     #'\n CIL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])))) +
     '\n CIL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])))) +
     #'\n CIL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])))) +
      '\n MDN: ' + str(np.mean(np.abs(true_y - pred_mdn))) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc))) +
     '\n CIL No Bayes: ' + str(np.mean(np.abs(true_y - y_pred))) +
     '\n CIL Benchmark: ' + str(np.mean(np.abs(true_y - 0))) +
      
      '\nAccuracy I:' +
     '\n CIL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) <= 6)) +
     #'\n CIL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) <= 6)) +
     '\n CIL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) <= 6)) +
     #'\n CIL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])) <= 6)) +
     '\n MDN: ' + str(np.mean(np.abs(true_y - pred_mdn) <= 6)) +
     '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc) <= 6)) +
      '\n CIL No Bayes: ' + str(np.mean(np.abs(true_y - y_pred) <= 6)) +
     '\n CIL Benchmark: ' + str(np.mean(np.abs(true_y - 0) <= 6)) +
    
      '\nAccuracy II:' +
     '\n CIL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) <= 2)) +
     #'\n CIL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) <= 2)) +
     '\n CIL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) <= 2)) +
     #'\n CIL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])) <= 2)) +
    '\n MDN: ' + str(np.mean(np.abs(true_y - pred_mdn) <= 2)) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc) <= 2)) +
     '\n CIL No Bayes: ' + str(np.mean(np.abs(true_y - y_pred) <= 2)) +
     '\n CIL Benchmark: ' + str(np.mean(np.abs(true_y - 0) <= 2)) 
     )

In [ ]:
df = pd.DataFrame({'va_ridge' : result_va_ridge['densities'],
                 'hmc_ridge' : result_hmc_ridge['densities'],
                 'va_horse' : result_va_horse['densities'],
                 'hmc_horse' : result_hmc_horse['densities']})

In [ ]:
df.to_csv('../../../../data/commaai/predictions/cil/densities.csv')